In [10]:
import numpy as np
import pandas as pd
import datetime as dt
parser = lambda Date: pd.datetime.strptime(Date, '%a, %b %d, %Y')
dataset = pd.read_csv('C:/Users/Administrator/Desktop/NBA.csv', encoding='ANSI',parse_dates=['Date'], date_parser=parser,usecols=[0,2,3,4,5,6,7,8])
dataset.columns=['Date', 'Visitor Team', 'VisitorPts', 'Home Team', 'HomePts','Score Type', 'OT?', 'Notes']

In [11]:
dataset['HomeWin'] = dataset['VisitorPts'] < dataset['HomePts']
dataset['HomeLastWin'] = np.NAN
dataset['VisitorLastWin'] = np.NAN
y_true = dataset['HomeWin'].values    #类别数据，sklearn可以直接读取
from collections import defaultdict
won_last = defaultdict(int)
for index, row in dataset.sort_values('Date').iterrows():
    home_team = row['Home Team']
    visitor_team = row['Visitor Team']
    row['HomeLastWin'] = won_last[home_team]
    row["VisitorLastWin"] = won_last[visitor_team]
    dataset.iloc[index] = row 
    won_last[home_team] = row['HomeWin']
    won_last[visitor_team] = not row['HomeWin']
#dataset.iloc[20:25]

In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
clf = DecisionTreeClassifier(random_state=14)
X_previouswins = dataset[['HomeLastWin', 'VisitorLastWin']].values
scores = cross_val_score(clf, X_previouswins, y_true, scoring='accuracy')
print('Accuracy: {0:.1f}%'.format(np.mean(scores)*100))

Accuracy: 57.5%


In [13]:
standings = pd.read_csv('C:/Users/Administrator/Desktop/standings.csv', encoding='ANSI',skiprows=[0,])
#standings.head()

In [14]:
#用主客场和排名分类
dataset['HomeTeamRanksHigher']=0
for index, row in dataset.iterrows():
    home_team = row['Home Team']
    visitor_team = row['Visitor Team'] 
    if home_team == 'New Orleans Pelicans':
        home_team = 'New Orleans Hornets'
    elif visitor_team == 'New Orleans Pelicans':
        visitor_team = 'New Orleans Hornets'
    home_rank = standings[standings['Team'] == home_team]['Rk'].values[0]
    #print(home_rank,',',visitor_team, index)
    visitor_rank = standings[standings['Team'] == visitor_team]['Rk'].values[0]
    #print(index, visitor_team,visitor_rank)

        
    row['HomeTeamRanksHigher'] = int(home_rank > visitor_rank)
    dataset.iloc[index] = row
X_homehigher = dataset[['HomeLastWin', 'VisitorLastWin', 'HomeTeamRanksHigher']].values
scores = cross_val_score(clf, X_homehigher, y_true, scoring='accuracy')
print('Accuracy: {0:.1f}%'.format(np.mean(scores)*100))

Accuracy: 59.7%


In [15]:
#用排名和上场赢家分类
last_match_winner = defaultdict(int)    #为什么这里是int，而不是str
dataset['HomeTeamWonLast'] = 0 
for index, row in dataset.iterrows():
    home_team = row['Home Team']
    visitor_team = row['Visitor Team']
    teams = tuple(sorted([home_team, visitor_team]))
    row['HomeTeamWonLast'] = 1 if last_match_winner[teams] == row['Home Team'] else 0
    dataset.iloc[index] =row
    winner = row['Home Team'] if row['HomeWin'] else row['Visitor Team']
    last_match_winner[teams] = winner
    
X_lastwinner = dataset[['HomeTeamRanksHigher', 'HomeTeamWonLast']].values
clf = DecisionTreeClassifier(random_state=14)
scores = cross_val_score(clf, X_lastwinner, y_true, scoring='accuracy')
print('Accuracy: {0:.1f}%'.format(np.mean(scores)*100))

Accuracy: 57.8%


In [16]:
#team names were transformed into num_values variables
from sklearn.preprocessing import LabelEncoder
encoding = LabelEncoder()
encoding.fit(dataset['Home Team'].values)
home_teams = encoding.transform(dataset['Home Team'].values)
#print(home_teams)
visitor_teams = encoding.transform(dataset['Visitor Team'].values)
#print(visitor_teams)
X_teams = np.vstack([home_teams, visitor_teams]).T
#print(X_teams)
# num_values were transformed into binary
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder()
X_team_expended = onehot.fit_transform(X_teams).todense()
#print(X_team_expended)
clf = DecisionTreeClassifier(random_state=14)
scores = cross_val_score(clf, X_team_expended, y_true, scoring='accuracy')
print('Accuracy: {0:.1f}%'.format(np.mean(scores)*100))

Accuracy: 59.5%


In [17]:
# using Randomforestclassifier
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=14)
scores = cross_val_score(clf, X_teams, y_true, scoring='accuracy')
print('Accuracy: {0:.1f}%'.format(np.mean(scores)*100))

Accuracy: 58.4%


In [10]:
X_all = np.hstack([X_homehigher, X_teams])
clf = RandomForestClassifier(random_state=14)
scores = cross_val_score(clf, X_all, y_true, scoring='accuracy')
print('Accuracy: {0:.1f}%'.format(np.mean(scores)*100))

Accuracy: 57.6%


In [35]:
#gridsearchCV looking for best parameters
from sklearn.model_selection import GridSearchCV
parameter_space = {'max_features':[2, 3, 4, 5, 'auto'], 'n_estimators': [100,], 'criterion': ['gini', 'entropy'], 'min_samples_leaf': [2, 4, 6]}
clf = RandomForestClassifier(random_state=14)
grid = GridSearchCV(clf, parameter_space)
grid.fit(X_all, y_true)
print('Accuracy: {0:.1f}%'.format(grid.best_score_ *100))

Accuracy: 62.9%


In [36]:
print(grid.best_estimator_)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=5, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=2,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=14,
            verbose=0, warm_start=False)


In [18]:
X_teams.shape()

TypeError: 'tuple' object is not callable